<center>
<img src="../img/python_theme.png">
# Майнор "Интеллектуальный анализ данных" 
# Курс "Введение в программирование"
<img src="../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: преподаватель ФКН НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-Share Alike 4.0</a>. Можно использовать в любых целях, но с обязательным упоминанием автора курса и аффилиации.

# Семинар 7. Дополнительная часть. Пример реальной задачи по парсингу данных

Допустим, в кассе "Яндекс" есть счета, выставленные клиентам магазина, и соттветсвующие платежи. Платежи выставляются на конкретный e-mail и имеют ID платежа. Все почтовые ящики уникальны, на один ящик может быть послано несколько платежей с разными ID (обязаьтельно разными) и суммой (необязательно разными).

<img src="../img/kassa_bills.png">

По выставленным платежам есть реальные оплаты. ID платежа и сумма.

<img src="../img/kassa_payments.png">

Как нам составить таблицу почтовых ящиков с реально оплаченными счетами, чтоб понять, кто сколько заплатил в сумме?

Создадим функцию, которая считывает файл с данными о счетах и возвращает словарь словарей  вида {e-mail: {trans_id1: amount1, trans_id2: amount2}}

In [1]:
BILL_ID_PREFIX = "48241-"

def parse_bills(input_path):
    bills_dict = {}
    with open(input_path) as bills:
        for line in bills:
            if BILL_ID_PREFIX in line:
                trans_id = int(line.strip().split(BILL_ID_PREFIX)[1])
                email_id = bills.readline().strip()
                amount = int(bills.readline().strip()\
                             .split(',')[0].replace(' ',''))
                if email_id not in bills_dict:
                    bills_dict[email_id] = {trans_id: amount}
                else:
                    bills_dict[email_id][trans_id] = amount
    return bills_dict

In [2]:
print(parse_bills("../data/kassa_bills.txt"))

{'kushchev.va@vtb24.ru': {10: 42}, 'moemailn@gmail.com': {17: 32}, 'bobrovviktor@gmail.com': {36: 12, 47: 10}, 'kirgush@gmail.com': {4: 20}, 'dkishmareishvili@gmail.com': {43: 30, 63: 2}, 'vbokr@mail.ru': {57: 10, 58: 10, 9: 20}, 'diana.sungatullina@gmail.com': {38: 42}, 'bellmsk@gmail.com': {18: 42}, 'info@lifefilm.ru': {21: 22}, 'korzhikdmitry@gmail.com': {83: 12}, 'seregalu@yandex.ru': {23: 32}, 'susumanin@gmail.com': {34: 22}, 'elena.fomenko8@gmail.com': {65: 10, 66: 10, 51: 20}, '441976@inbox.ru': {24: 12}, 'khusnutdinova.e@gmail.com': {3: 10}, 'a-karelsky@hotmail.com': {48: 10}, 'pavelpakhomov@gmail.com': {33: 22, 69: 10}, 'intendia@gmail.com': {27: 10}, 'aloha@74.ru': {29: 12}, 'Hellboysis666@gmail.com': {37: 42}, 'dimkat@gmail.com': {6: 12}, 'perestrelka@gmail.com': {54: 20}, 'arsen.nafka@gmail.com': {71: 10}, 'alexhaker@yahoo.co.uk': {7: 42}, 'sergey.salnikov.dev@yandex.ru': {14: 42, 60: 10, 77: 10, 62: 10, 79: 12}, 'dmitry.okhezin@gmail.com': {44: 42}, 'pavel7ovchinnikov@gmai

Теперь попроще - обработаем файл с платежами. Создадим функцию, которая считывает файл с платежами и возвращает список ID платежей, по которым реально прошла оплата.

In [3]:
def parse_payments(input_path):
    paid_ids = []
    with open(input_path) as f:
        for line in f:
            if BILL_ID_PREFIX in line \
            and "ID" + BILL_ID_PREFIX not in line:
                paid_ids.append(line.strip().split('48241-')[1])
        paid_ids = [int(id) for id in paid_ids]
    return paid_ids

Вот ID счетов, по которым прошла оплата

In [4]:
print(parse_payments("../data/kassa_payments.txt"))

[82, 83, 80, 78, 73, 24, 77, 72, 76, 65, 75, 74, 66, 69, 13, 62, 64, 51, 61, 18, 2, 60, 59, 58, 57, 4, 20, 16, 5, 11, 55, 46, 29, 19, 48, 25, 9, 22, 27, 3, 47, 12, 15, 36, 45, 39, 33, 34, 6, 26]


Наконец, создадим финальный словарь

In [5]:
def paid_emails(bills_dict, paid_ids):
    paid_emails_dic = {}
    for email_id in bills_dict:
        for trans_id in bills_dict[email_id]:
            if trans_id in paid_ids:
                if email_id not in paid_emails_dic:
                    paid_emails_dic[email_id] = [[trans_id], 
                                          bills_dict[email_id][trans_id]]
                else:
                    new_ids, new_amount = paid_emails_dic[email_id][0], \
                                            paid_emails_dic[email_id][1]
                    new_ids.extend([trans_id])
                    new_amount += bills_dict[email_id][trans_id]
                    paid_emails_dic[email_id] = [new_ids, new_amount]
    return paid_emails_dic

In [6]:
bills_dict = parse_bills("../data/kassa_bills.txt")
paid_ids = parse_payments("../data/kassa_payments.txt")

print(paid_emails(bills_dict, paid_ids))

{'roslov.anton@gmail.com': [[2], 10], 'roman.zakharovp@gmail.com': [[13], 22], 'khusnutdinova.e@gmail.com': [[3], 10], 'dimkat@gmail.com': [[6], 12], 'bobrovviktor@gmail.com': [[36, 47], 22], 'antonkiselev@me.com': [[59], 10], 'vbokr@mail.ru': [[57, 58, 9], 40], 'kkrasnoschekov@genplanmos.ru': [[72, 73], 22], 'bellmsk@gmail.com': [[18], 42], 'ni-ko-lay@mail.ru': [[82, 61], 22], 'frolova.victory@gmail.com': [[74, 76], 22], '441976@inbox.ru': [[24], 12], 'elena.fomenko8@gmail.com': [[65, 66, 51], 40], 'belindr@mail.ru': [[12], 22], 'susumanin@gmail.com': [[34], 22], 'a.l.shevchenko.ua@gmail.com': [[5], 22], 'belskikh.aleksandr@gmail.com': [[45], 42], 'a-karelsky@hotmail.com': [[48], 10], 'sergey.salnikov.dev@yandex.ru': [[60, 77, 62], 30], 'intendia@gmail.com': [[27], 10], 'bendyna.ivan@gmail.com': [[26], 32], 'pro-maker@ya.ru': [[22], 42], 'aloha@74.ru': [[29], 12], 'murashkin.alexandr@gmail.com': [[16], 12], 'andryuha-v@mail.ru': [[80, 64, 75], 32], 'atselikov@gmail.com': [[39], 20], '

Напоследок запишем полученные данный в csv-файл, который потом можно редактировать в Excel или текстовом редакторе

In [7]:
def write_to_csv(output_path, paid_emails_dic):
    with open(output_path,'w') as paid_file:
        for email_id in paid_emails_dic:
            paid_file.write('{0};{1};{2}\n'\
                            .format(email_id, 
                                    paid_emails_dic[email_id][0], 
                                    paid_emails_dic[email_id][1]))

In [8]:
paid_emails_dic = paid_emails(bills_dict, paid_ids)
write_to_csv("paid_emails.csv", paid_emails_dic)

Получили на выходе scv-файл, из которого легко сделать такую таблицу:

<img src="../img/paid_emails_output.png">